In [1]:
import pandas as pd
import numpy as np
from math import sqrt

In [2]:
def inteName(i):
    if i==0:
        return "1-9"
    elif i<10:
        return str(int(i*10))+"-"+str(int(i*10+9))
    elif i>=50:
        return 'sup500'
    else :
        return str(int(i*10))+"-"+str(int(i*10+49))

In [3]:
dicoDf={}
for i in range(10) :
    dicoDf[i]=pd.read_csv('data/prediction_fold_'+str(i)+'.csv')

In [4]:
listDF=dicoDf.values()

In [5]:
dicoDf[0].head()

,userId,movieId,rating,Predicted ratings BaselineOnly,Predicted ratings SVD,Predicted ratings NMF
0,1,0000000116,4.0,4.072118,4.095852,1.000000
1,2,1477817409,4.0,4.203534,4.200615,2.121743
2,3,0345367413,4.0,3.441428,3.391529,3.447826
3,7,0000013714,4.0,4.382731,4.405936,3.964205
4,9,0882703803,5.0,4.641738,4.874347,5.000000


In [6]:
algoListe=["BaselineOnly", "SVD", "NMF", "SlopeOne", 
           "CoClustering", "NormalPredictor", "SVDpp",
           "KNNBasic", "KNNWithMeans", "KNNBaseline", "KNNWithZScore"]

In [7]:
nbAlgo=2

In [8]:
for df in listDF:
    for name in algoListe[:nbAlgo]:
        df["error "+name]=abs(df["rating"]-df["Predicted ratings "+name])

In [9]:
ncol=list(dicoDf[0])

In [10]:
errS=6

In [11]:
ncol

['userId',
 'movieId',
 'rating',
 'Predicted ratings BaselineOnly',
 'Predicted ratings SVD',
 'Predicted ratings NMF',
 'error BaselineOnly',
 'error SVD']

In [12]:
d={}
ar=df.to_numpy()

In [13]:
def rmse(l):
    s=0
    for i in l:
        s+=i*i
    return sqrt(s/len(l))

def avg(l):
    s=0
    for i in l:
        s+=i
    return s/len(l)

In [14]:
nameColumms=['UserId','Number of rated movies in the training dataset','Number of rated movies in the testing dataset']
for name in algoListe[:nbAlgo] :
    nameColumms.append('RMSE '+name+', per user')
for name in algoListe[:nbAlgo] :
    nameColumms.append('MAE '+name+', per user')

In [17]:
listDfRes=list()
for fold in range(10):
    print('--== ', fold,' ==--')
    d={}
    ar=dicoDf[fold].to_numpy()
    t=len(ar)
    for i in range(t) :
        ID=ar[i,0]
        if ID not in d :
            d[ID]={}
            for j in range(nbAlgo) :
                d[ID][j]=[ar[i,j+errS]]
        else :
            for j in range(nbAlgo) :
                d[ID][j].append(ar[i,j+errS])

    dfDico=pd.read_csv('dataCV3/user-nbRating_fold_'+str(fold)+'.csv')
    dRatingsTrain={}
    dRatingsTest={}
    for index, row in dfDico.iterrows():
        dRatingsTrain[row['userId']]=row['NBRatingTrain']
        dRatingsTest[row['userId']]=row['NBRatingTest']
    del dfDico

    d2={}
    for i in d:
        d2[i]={}
        for j in range(nbAlgo):
            d2[i][j]=rmse(d[i][j])

    d3={}
    for i in d:
        d3[i]={}
        for j in range(nbAlgo):
            d3[i][j]=avg(d[i][j])

    listAR=list()
    for i in d2 :
        listAR.append(np.array([[int(i),int(dRatingsTrain[i]), int(dRatingsTest[i]),
                                 d2[i][0], d2[i][1], 
                                 d3[i][0], d3[i][1]]]))
    arRes=np.concatenate(listAR)
    dfRes=pd.DataFrame({nameColumms[0]:arRes[:,0],nameColumms[1]:arRes[:,1],nameColumms[2]:arRes[:,2],nameColumms[3]:arRes[:,3],
                        nameColumms[4]:arRes[:,4],nameColumms[5]:arRes[:,5],nameColumms[6]:arRes[:,6]})

    dfRes.to_csv('data/uRMSE_uMAE_fold_'+str(fold)+'.csv',index=False)
    listDfRes.append(dfRes)

--==  0  ==--
--==  1  ==--


IndexError: index 7 is out of bounds for axis 1 with size 7

In [ ]:
dfRes=pd.concat(listDfRes)
dfRes.to_csv('data/uRMSE_uMAE_full.csv',index=False)

In [ ]:
dfRes.head()

In [19]:
del d,d3,d2,df

In [20]:
len(dfRes)

6239080

In [21]:
cpt=0
for df in listDF :
    print(len(df))
    cpt+=len(df)

    
print("tot = ",cpt)

825100
825100
825100
825100
903951
825100
825100
825100
825100
903951
tot =  8408702


In [22]:
cpt2=0
lres=list()
for fold in range(10):
    dfTemp=pd.read_csv('data/uRMSE_uMAE_fold_'+str(fold)+'.csv')
    print(len(dfTemp))
    cpt2+=len(dfTemp)
    lres.append(dfTemp)
    
print("tot = ",cpt2)

623908
623908
623908
623908
623908
623908
623908
623908
623908
623908
tot =  6239080


In [23]:
ar=dfRes.to_numpy()

In [24]:
t=len(ar)
d={}
for i in range(t) :
    ID=ar[i,1]
    if ID not in d:
        d[ID]={}
        for j in range(nbAlgo):
            d[ID][j]=[ar[i,3+j]]
    else :
        for j in range(nbAlgo):
            d[ID][j].append(ar[i,3+j])

In [25]:
d2={}
for i in d:
    d2[i]={}
    for j in range(nbAlgo):
        d2[i][j]=avg(d[i][j])

In [27]:
listAR=list()
for i in d2 :
    listAR.append(np.array([[int(i), d2[i][0], d2[i][1], d2[i][2]]]))

In [28]:
arRes=np.concatenate(listAR)

In [29]:
nameColumms=['Number of rated movies']
for name in algoListe[:nbAlgo] :
    nameColumms.append('avg RMSE '+name)

In [30]:
dfRes=pd.DataFrame({nameColumms[0]:arRes[:,0],nameColumms[1]:arRes[:,1],nameColumms[2]:arRes[:,2],nameColumms[3]:arRes[:,3]})
                    #nameColumms[4]:arRes[:,4],nameColumms[5]:arRes[:,5],nameColumms[6]:arRes[:,6]})

In [31]:
dfRes.head()

,Number of rated movies,avg RMSE BaselineOnly,avg RMSE SVD,avg RMSE NMF
0,2.0,0.819167,0.809349,0.982313
1,30.0,0.953808,0.953629,1.037730
2,6.0,0.803341,0.786319,0.870056
3,1.0,0.838791,0.831812,1.237366
4,32.0,0.933214,0.934288,1.031612


In [32]:
dfRes.to_csv('data/RMSEavg_cv.csv',index=False)